In [48]:
import pathlib
import geopandas as gpd
import numpy as np
import pandas as pd

import shapely

In [8]:
src_dir = pathlib.Path('~').expanduser() / 'data'
dataset_filename = src_dir / 'ivs/ivs-2024-geocoded-sample.gpkg'
hubs_filename = src_dir / 'ivs/hubs.geojson'

ivs_gdf = gpd.read_file(dataset_filename)
hubs = gpd.read_file(hubs_filename)

In [44]:
def get_src_point(line):
    src = list(line.coords)[0]
    src = shapely.Point(src)
    return src

def get_target_point(line):
    target = list(line.coords)[-1]
    target = shapely.Point(target)
    return target

In [69]:
ivs_gdf['time'] = pd.to_datetime(ivs_gdf['v05_06_begindt_evenement_iso'], format='ISO8601')
ivs_gdf = ivs_gdf[(ivs_gdf['Weeknr'] == 38) & (ivs_gdf['Jaar'] == 2022)]
ivs_gdf = ivs_gdf[ivs_gdf['geometry'] != None]
ivs_gdf['src_geometry'] = ivs_gdf['geometry'].apply(get_src_point)
ivs_gdf['target_geometry'] = ivs_gdf['geometry'].apply(get_target_point)

C:\Users\hemert\AppData\Local\Temp\ipykernel_23852\3326428940.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  ivs_gdf['time'] = pd.to_datetime(ivs_gdf['v05_06_begindt_evenement_iso'], format='ISO8601')


In [70]:
ivs_gdf

,Jaarmaand,Jaar,Maand,Weeknr,v05_06_begindt_evenement_iso,v05_06_Begindt_evenement,UNLO_herkomst,UNLO_bestemming,v15_1_Scheepstype_RWS,SK_CODE,v18_Laadvermogen,v28_Beladingscode,v38_Vervoerd_gewicht,v30_4_Containers_TEU_S,nstr_nw,nst2007_nw,geometry,src_geometry,target_geometry,time
319,2209,2022,9,38,2022-09-20T10:00:00+02:00,20 september 2022 10:00:00 uur,NLCUY,NLMAA,21,BI,1280.0,1,NaN,0.0,,,"LINESTRING (5.88333 51.73333, 6.03333 51.56667)",POINT (5.88333 51.73333),POINT (6.03333 51.56667),2022-09-20 10:00:00+02:00
322,2209,2022,9,38,2022-09-22T13:00:00+02:00,22 september 2022 13:00:00 uur,NLRTM,NLBNB,1,M8,3200.0,7,255808.0,84.0,9,12.1,"LINESTRING (4.50000 51.91667, 6.65719 52.31160)",POINT (4.50000 51.91667),POINT (6.65719 52.31160),2022-09-22 13:00:00+02:00
761,2209,2022,9,38,2022-09-23T08:00:00+02:00,23 september 2022 08:00:00 uur,NLVLI,BEANR,2,M8,3050.0,7,2565000.0,0.0,3,7.4,"LINESTRING (3.70000 51.45000, 4.41667 51.21667)",POINT (3.70000 51.45000),POINT (4.41667 51.21667),2022-09-23 08:00:00+02:00
862,2209,2022,9,38,2022-09-25T14:00:00+02:00,25 september 2022 14:00:00 uur,DESGW,NLRTM,24,BII-4,11510.0,5,0.0,0.0,9,16,"LINESTRING (6.73333 51.50000, 4.50000 51.91667)",POINT (6.73333 51.50000),POINT (4.50000 51.91667),2022-09-25 14:00:00+02:00
1692,2209,2022,9,38,2022-09-22T01:00:00+02:00,22 september 2022 01:00:00 uur,BEANR,NLRTM,7,C3b,6920.0,1,NaN,0.0,,,"LINESTRING (4.41667 51.21667, 4.50000 51.91667)",POINT (4.41667 51.21667),POINT (4.50000 51.91667),2022-09-22 01:00:00+02:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96799,2209,2022,9,38,2022-09-24T10:00:00+02:00,24 september 2022 10:00:00 uur,BEGNE,BEANR,2,M8,2850.0,5,0.0,0.0,8,8.5,"LINESTRING (3.71667 51.05000, 4.41667 51.21667)",POINT (3.71667 51.05000),POINT (4.41667 51.21667),2022-09-24 10:00:00+02:00
97189,2209,2022,9,38,2022-09-22T11:00:00+02:00,22 september 2022 11:00:00 uur,DEDUI,NLAMS,2,M8,2260.0,7,767123.0,0.0,9,20.0,"LINESTRING (6.75000 51.46667, 4.81667 52.40000)",POINT (6.75000 51.46667),POINT (4.81667 52.40000),2022-09-22 11:00:00+02:00
97267,2209,2022,9,38,2022-09-20T06:00:00+02:00,20 september 2022 06:00:00 uur,NLLIN,BEKSN,21,BI,1570.0,7,1350000.0,0.0,6,3.5,"LINESTRING (5.93333 51.15000, 5.81667 51.15000)",POINT (5.93333 51.15000),POINT (5.81667 51.15000),2022-09-20 06:00:00+02:00
97829,2209,2022,9,38,2022-09-22T12:00:00+02:00,22 september 2022 12:00:00 uur,NLDOR,NLMOE,31,BI,2020.0,3,0.0,0.0,8,8.5,"LINESTRING (4.63333 51.78333, 4.56667 51.68333)",POINT (4.63333 51.78333),POINT (4.56667 51.68333),2022-09-22 12:00:00+02:00


In [62]:
routes = pd.merge(hubs, hubs, how='cross')
routes = routes[routes['name_x'] != routes['name_y']]
routes

,name_x,geometry_x,name_y,geometry_y
1,Nijmegen,"POLYGON ((5.78912 51.88572, 5.76470 51.85063, ...",Rotterdam,"POLYGON ((4.56209 51.91397, 4.48988 51.93330, ..."
2,Nijmegen,"POLYGON ((5.78912 51.88572, 5.76470 51.85063, ...",Duisburg,"POLYGON ((6.85010 51.45693, 6.72340 51.53244, ..."
3,Rotterdam,"POLYGON ((4.56209 51.91397, 4.48988 51.93330, ...",Nijmegen,"POLYGON ((5.78912 51.88572, 5.76470 51.85063, ..."
5,Rotterdam,"POLYGON ((4.56209 51.91397, 4.48988 51.93330, ...",Duisburg,"POLYGON ((6.85010 51.45693, 6.72340 51.53244, ..."
6,Duisburg,"POLYGON ((6.85010 51.45693, 6.72340 51.53244, ...",Nijmegen,"POLYGON ((5.78912 51.88572, 5.76470 51.85063, ..."
7,Duisburg,"POLYGON ((6.85010 51.45693, 6.72340 51.53244, ...",Rotterdam,"POLYGON ((4.56209 51.91397, 4.48988 51.93330, ..."


In [72]:
row = routes.iloc[0]
index = np.logical_and.reduce([
    ivs_gdf['src_geometry'].apply(lambda geometry: geometry.intersects(row['geometry_x'])),
    ivs_gdf['target_geometry'].apply(lambda geometry: geometry.intersects(row['geometry_y']))
])

In [73]:
ivs_gdf[index]

,Jaarmaand,Jaar,Maand,Weeknr,v05_06_begindt_evenement_iso,v05_06_Begindt_evenement,UNLO_herkomst,UNLO_bestemming,v15_1_Scheepstype_RWS,SK_CODE,v18_Laadvermogen,v28_Beladingscode,v38_Vervoerd_gewicht,v30_4_Containers_TEU_S,nstr_nw,nst2007_nw,geometry,src_geometry,target_geometry,time
